In [1]:
import requests
import json
import datetime
import pandas as pd
import pytz

class FastGenie:
    
    def __init__(self, location, start_date):
        street, city, state, zipcode = location.split(',')
        self.street = street.strip()
        self.city = city.strip()
        self.state = state.strip()
        self.zipcode = zipcode.strip()
        self.start_date = start_date
        self.description = "This is very simple tool to generate a time table for nerdy Baha'is during the fast."
        
    def geo_api(self):
        GEO_API = 'https://geocoding.geo.census.gov/geocoder/geographies/address?'
        api_out = requests.get(GEO_API, params={'street': self.street,
            'city': self.city,
            'state': self.state,
            'zip': self.zipcode,
            'benchmark': 'Public_AR_Census2010',
            'vintage': 'Census2010_Census2010',
            'layers': 14,
            'format': 'json',}).json()['result']
        
        lat = api_out['addressMatches'][0]['geographies']['Census Blocks'][0]['CENTLAT']
        lng = api_out['addressMatches'][0]['geographies']['Census Blocks'][0]['CENTLON']
        
        return lat,lng
    
    def get_sun_schedule(self):
        lat,lng = self.geo_api()
        SUNRISE_API = 'https://api.sunrise-sunset.org/json?'
        series_date = pd.date_range(self.start_date, freq='D', periods=21)
        datelist = series_date.astype(str)
        
        output = list()
        
        # Easier to loop to collect the date, too
        for date in datelist:
            api_out = requests.get(SUNRISE_API, params={'lat': lat,
                                                        'lng': lng,
                                                        'date': date,
                                                        'formatted': 0}).json()['results']
            api_out['date'] = date
            api_out['day_of_week'] = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')
            output.append(api_out)
        return output

    def get_time_table(self):
        good_data = self.get_sun_schedule()
        df = pd.DataFrame(good_data)
        df = df[['day_of_week','date','sunrise','sunset']]
        
        # Set timezone
        tz = pytz.timezone('US/Pacific')
        df['sunrise'] = pd.to_datetime(df['sunrise']).dt.tz_convert('US/Pacific')
        df['sunset'] = pd.to_datetime(df['sunset']).dt.tz_convert('US/Pacific')
        
        # Beautify
        df['sunrise'] = df['sunrise'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        df['sunset'] = df['sunset'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        
        return df

## Enter your address and start date, and run this baby!

In [2]:
YOUR_ADDRESS = '1451 Eden Ave,San Jose,CA,95117'
FIRST_DAY_OF_FAST = '2020-03-01'
FastPrinter = FastGenie(YOUR_ADDRESS,FIRST_DAY_OF_FAST)
FastPrinter.get_time_table()

,day_of_week,date,sunrise,sunset
0,Sunday,2020-03-01,06:36:56 AM,06:02:48 PM
1,Monday,2020-03-02,06:35:32 AM,06:03:46 PM
2,Tuesday,2020-03-03,06:34:08 AM,06:04:45 PM
3,Wednesday,2020-03-04,06:32:43 AM,06:05:43 PM
4,Thursday,2020-03-05,06:31:18 AM,06:06:40 PM
5,Friday,2020-03-06,06:29:52 AM,06:07:37 PM
6,Saturday,2020-03-07,06:28:25 AM,06:08:35 PM
7,Sunday,2020-03-08,07:26:59 AM,07:09:31 PM
8,Monday,2020-03-09,07:25:31 AM,07:10:28 PM
9,Tuesday,2020-03-10,07:24:03 AM,07:11:24 PM
